In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
from chromalab.observer import Observer, Cone, transformToChromaticity, getHeringMatrix
from chromalab.spectra import Spectra, Illuminant, convert_refs_to_spectras
from chromalab.maxbasis import MaxBasis
from chromalab.visualizer import PSWrapper, DisplayBasisType, exportAndPlay, easeFunction
from chromalab.animation import Animation

objc[5740]: Class GLFWApplicationDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ba386e28) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2d62366f0). One of the two will be used. Which one is undefined.
objc[5740]: Class GLFWWindowDelegate is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ba386db0) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/polyscope_bindings.cpython-311-darwin.so (0x2d6236718). One of the two will be used. Which one is undefined.
objc[5740]: Class GLFWContentView is implemented in both /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-packages/open3d/cpu/pybind.cpython-311-darwin.so (0x2ba386e50) and /Users/jessicalee/anaconda3/envs/chromalab/lib/python3.11/site-pac

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib ipympl

In [4]:
wavelengths1 = np.arange(390, 701, 1)
wavelengths5 = np.arange(400, 701, 5)
wavelengths10 = np.arange(400, 701, 10)

In [5]:
wavelengths = np.arange(400, 710, 5)
d65 = Illuminant.get("D65")

In [6]:
trichromat = Observer.trichromat(wavelengths=wavelengths5, illuminant=d65) 
maxbasis_tri = MaxBasis(trichromat, verbose=True)
viz = PSWrapper(trichromat, maxbasis_tri, itemsToDisplay=PSWrapper.ItemsToDisplay.BOTH, displayBasis=DisplayBasisType.MAXBASIS, verbose=True)

100%|██████████| 58/58 [00:00<00:00, 646.12it/s]
/Users/jessicalee/Projects/generalized-colorimetry/code/ChromaLab/chromalab/spectra.py:36: UserWarning: Data has values not between 0 and 1. Clipping.
  warnings.warn("Data has values not between 0 and 1. Clipping.")


[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 83.1


100%|██████████| 3660/3660 [00:08<00:00, 419.29it/s]


In [7]:
names = ['mesh', 'lattice', 'qarrow', 'lumAxis']

In [8]:
# general settings - RUN THIS CELL TO RESET AN ANIMATION RUN
viz.ps.set_ground_plane_mode("none")
offset = 0 # start at 0 frames indexing

In [9]:
# render all objects
viz.renderObjectsPS(mesh_alpha=0.8, lattice_alpha=1)

# transform to correct dir
mat = np.eye(4)
mat[:3, :3] = viz.HMatrix[::-1]
viz.ps.get_surface_mesh("mesh").set_transform(mat)
viz.ps.get_surface_mesh("lattice").set_transform(mat)

In [10]:
def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

In [11]:
transforms_LMS, transforms_RGB = viz._getIntermediateTransformsLMSRGB(2)
HMat = viz._getmatrixBasisToLum()

ps_mesh = viz.ps.get_surface_mesh("mesh")
names += viz._getCoordBasis("RGB_coords", ((HMat@np.eye(4))[:3, :3]).T)
names += viz._getCoordBasis("LMS_coords", ((HMat@normalized(transforms_LMS[-1]))[:3, :3]).T)
names += viz.renderLuminanceAxis()

In [12]:
long_frames = 90
medium_frames = 60
short_frames = 30

In [13]:
types =  ['surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh', 'surface_mesh']
map_name_type = {name:typ for name, typ in zip(names, types)}
anim = Animation(viz, map_name_type)

In [14]:
filename = "./outputs/obs-seq-siggraph.mp4"
anim.set_enabled('lattice', False)
anim.set_enabled('lumAxis', False)
anim.set_enabled('RGB_coords', False)
anim.set_enabled('mesh', True)
fd = viz.openVideo(filename)

In [15]:
r, theta, phi = 4, 90, 270
origin = np.array([0, 0, np.sqrt(3)/2])
offset = 0

In [16]:
# squash along L Axis
def Squash(i):
    viz.renderFlattenExistingMesh(mesh_alpha=0.8, height=1 - (i/short_frames), basis=viz.ps.get_surface_mesh("mesh").get_transform()[:3, :3])

rotAroundZ, phi = anim.RotateAroundZ(long_frames, r, theta, [0, phi], lookAt=origin)
offset = viz.renderVideo(rotAroundZ, fd, long_frames, offset)

anim.set_enabled("lumAxis", True)
fadeInLum = anim.FadeIn("lumAxis", medium_frames, [0.3, 1])
fadeOutMesh = anim.FadeOut("mesh", medium_frames, [0.3, 0.8])
rotAroundZ, phi = anim.RotateAroundZ(medium_frames, r, theta, [phi, phi +45], lookAt=origin)
offset = viz.renderVideo(anim.concatFns([fadeInLum, fadeOutMesh, rotAroundZ]), fd, medium_frames, offset)

fadeInMesh = anim.FadeIn("mesh", medium_frames, [0.3, 0.8])
rotAroundZ, phi = anim.RotateAroundZ(medium_frames, r, theta, [phi, phi +45], lookAt=origin)
offset = viz.renderVideo(anim.concatFns([fadeInMesh, rotAroundZ]), fd, medium_frames, offset)

anim.set_enabled("mesh", False)
offset = viz.renderVideo(Squash, fd, short_frames, offset)

rotTheta = anim.RotateTheta(medium_frames, r, [90, 0], phi, lookAt=origin)
offset = viz.renderVideo(anim.concatFns([rotTheta]), fd, medium_frames, offset)

fadeOutLum = anim.FadeOut("lumAxis", short_frames, [0.3, 1], removeBefore=10)
offset = viz.renderVideo(anim.concatFns([fadeOutLum]), fd, short_frames, offset)

viz.closeVideo(fd)

ffmpeg version 7.0.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.1.0.2.5)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex 